In [117]:
from dataclasses import dataclass, field

@dataclass()
class Machine:
    button_a: tuple[int, int] = ()
    button_b: tuple[int, int] = ()

    prize: tuple[int, int] = ()
    max_a_presses: int = 0
    max_b_presses: int = 0

    def init_maxes(self):
        ax = self.button_a[0]
        ay = self.button_a[1]
        bx = self.button_b[0]
        by = self.button_b[1]

        px = self.prize[0]
        py = self.prize[1]

        max_ax = px // ax
        max_ay = py // ay
        max_bx = px // bx
        max_by = py // by
        self.max_a_presses = min(max_ax, max_ay)
        self.max_b_presses = min(max_bx, max_by)


def load_input() -> list[Machine]:
    with open("../../data/day13-input.txt") as f:
        machines: list[Machine] = []
        for group in f.read().split('\n\n'):
            machine = Machine()
            rules = group.split('\n')
            parts = rules[0].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_a = (int(parts[0]), int(parts[1]))
            parts = rules[1].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_b = (int(parts[0]), int(parts[1]))
            parts = rules[2].split(':')[1].replace('X=', '').replace('Y=', '').replace(' ', '').split(',')
            machine.prize = (int(parts[0]), int(parts[1]))
            machine.init_maxes()
            machines.append(machine)
    return machines


In [118]:
def vector_can_hit_target(vector: tuple[int, int], from_vector: tuple[int, int],
                          target_vector: tuple[int, int]) -> bool:
    """If so, then return how many steps it takes"""
    reference_vector = (target_vector[0] - from_vector[0], target_vector[1] - from_vector[1])
    ratio = (reference_vector[0] / reference_vector[1]).as_integer_ratio()
    if ratio == (vector[0] / vector[1]).as_integer_ratio():
        return reference_vector[0] // vector[0]
    return False


def find_steps_to_target(vector: tuple[int, int], target_vector: tuple[int, int], step: int, min_step: int,
                         max_step: int) -> int:
    """ Binary search, but it's not needed"""
    a, b = step * vector[0], step * vector[1]
    if a == target_vector[0] and b == target_vector[1]:
        #print("found:", step)
        return step
    elif a < target_vector[0]:
        # step forward by 1/2
        next_step = step + max(1, (max_step - step) // 2)
        #print("too little:", step, "next = ", next_step)
        return find_steps_to_target(vector, target_vector, next_step, step, max_step)
    else:
        # Step back by 1/2
        next_step = min_step + max(1, (step - min_step) // 2)
        #print("too big:", step, "next = ", next_step)
        return find_steps_to_target(vector, target_vector, next_step, min_step, step)


def solve_machine(machine: Machine):
    vec_a = machine.button_a
    vec_b = machine.button_b
    solutions = []

    # Check if you don't need to press B
    if vector_can_hit_target(vec_a, (0, 0), machine.prize):
        only_a_presses = vector_can_hit_target(vec_a, (0, 0), machine.prize)
        if only_a_presses is not False:
            solutions.append((only_a_presses, 0))

    for a_press in range(0, machine.max_a_presses):
        a_position = a_press * vec_a[0], a_press * vec_a[1]
        steps_to_target = vector_can_hit_target(vec_b, a_position, machine.prize)
        if steps_to_target is not False:
            solutions.append((a_press, steps_to_target))
    return solutions


def solve_problem():
    cost_a = 3
    cost_b = 1

    machines = load_input()
    total_cost = 0
    solutions = []
    for machine in machines:
        solution = solve_machine(machine)
        if solution:
            if len(solution) > 1:
                raise Exception("Not handled")
            a, b = solution[0]
            if 100 < a or 100 < b:
                print('skip:', solution[0])
                continue
            total_cost += a * cost_a + b * cost_b
            solutions.append((a, b, machine))

    return total_cost, solutions


total_cost_part1, part_1_solutions = solve_problem()
print(total_cost_part1)

30973


### Part 2

After solving part 1 the wrong way, I wrote the problem down on paper and realized this is a linear algebra problem.
It's asking if the solution to **v**A = **b** exists when **v** has integer components. Matrix, A, is the collection of
button vectors arranged as columns. Writing the equations on paper and arranging it as a matrix sets up the system of
equations to be reduced to upper right triangular form.

In [119]:
import sympy

def solve_part_2():
    cost_a = 3
    cost_b = 1

    offset = 10000000000000

    machines = load_input()
    total_cost = 0
    for machine in machines:
        machine.prize = (offset + machine.prize[0], offset + machine.prize[1])
        A = sympy.Matrix([
            [machine.button_a[0], machine.button_b[0]],
            [machine.button_a[1], machine.button_b[1]]
        ])
        b = sympy.Matrix(machine.prize)
        solution = A.solve(b)
        n, m = solution
        if n.is_integer and m.is_integer:
            total_cost += int(n) * cost_a + int(m) * cost_b
    return total_cost


print(solve_part_2())

95688837203288
